## R204433P Gurira Wesley Panashe 
Assignment 4 : 3. Clustering

In [13]:
import pandas as pd
import glob

# Path to my CSV files
path = 'Outputs/*.csv'

# Combining all CSV files into one DataFrame
all_files = glob.glob(path)
df_list = []

for filename in all_files:
    df = pd.read_csv(filename)
    df_list.append(df)

# Concatenating all DataFrames
combined_df = pd.concat(df_list, ignore_index=True)

In [22]:
# Viewing my combined dataset
combined_df

,Category,Content,Link,Source,Title,Unnamed: 5,Cluster
0,Business,As the world’s largest — and one of its most u...,https://www.aljazeera.com/economy/2024/5/8/a-n...,Al Jazeera,"Should India take from the rich, give the poor...",NaN,2
1,Business,Air safety officials in the United States are ...,https://www.aljazeera.com/economy/2024/5/7/us-...,Al Jazeera,US officials probe allegations Boeing workers ...,NaN,2
2,Business,– When Aamir Dhedhi took his mother to India i...,https://www.aljazeera.com/news/2024/5/8/pakist...,Al Jazeera,Pakistan bets on a cannabis high as its econom...,NaN,2
3,Business,Amazon has announced plans to invest $9bn in S...,https://www.aljazeera.com/economy/2024/5/7/ama...,Al Jazeera,Amazon to invest $9bn in Singapore to expand c...,NaN,2
4,Business,Elon Musk suggested testing Tesla’s full self-...,https://www.aljazeera.com/economy/2024/5/8/elo...,Al Jazeera,"Elon Musk floated robotaxi launch in China, Ch...",NaN,2
...,...,...,...,...,...,...,...
209,Sports,LIV Golf will stage its 2024 Individual Champi...,https://www.herald.co.zw/liv-golf-books-indivi...,The Herald,LIV Golf books Individual Championship for Sep...,NaN,0
210,Sports,London. — Bayer Leverkusen are gunning for rev...,https://www.herald.co.zw/leverkusen-looking-fo...,The Herald,Leverkusen looking for Roma revenge in Europa ...,NaN,0
211,Sports,"Blessing Malinganiza in Abu Dhabi, UAE\n\nWith...",https://www.herald.co.zw/lady-chevrons-pray-fo...,The Herald,Lady Chevrons pray for miracle,NaN,0
212,Sports,Zimpapers Sports Hub\n\nPlumtree . . . . . . ....,https://www.herald.co.zw/mbh-plumtree-take-cen...,The Herald,"MBH, Plumtree take centre stage",NaN,0


### saving combined data as a .pkl file

In [15]:
combined_df.to_pickle('combined_data.pkl')

### Preparing Data For Classification


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initializing a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=1000)

# Fitting and transforming the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_df['Content'].values.astype('U'))  # Assuming 'Content' column contains text


### Allying Clustering Algorithm

In [17]:
from sklearn.cluster import KMeans
import pickle

# Setting number of clusters
num_clusters = 4

# Initializing K-Means clustering
km = KMeans(n_clusters=num_clusters)

# Fittting K-Means clustering on the TF-IDF matrix
km.fit(tfidf_matrix)

#saving as a pickle 
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(km, f)

### Assigning clusters to Dataframe

In [18]:
# Adding cluster labels to DataFrame
combined_df['Cluster'] = km.labels_

#### Making sure Cluster is printed alongside the cartegory

In [19]:
cluster_categories = []
for cluster_id in range(num_clusters):
    cluster_articles = combined_df[combined_df['Cluster'] == cluster_id]
    most_common_category = cluster_articles['Category'].mode().values[0]
    cluster_categories.append(most_common_category)

### Testing clustering

In [20]:
# Printing number of articles in each cluster
print(combined_df['Cluster'].value_counts())

2    122
0     37
3     32
1     23
Name: Cluster, dtype: int64


In [21]:
for cluster_id in range(num_clusters):
    cluster_category = cluster_categories[cluster_id]
    print(f"Cluster {cluster_id}: {cluster_category}")
    print(combined_df[(combined_df['Cluster'] == cluster_id) & (combined_df['Category'] == cluster_category)].head(5))
    print("\n")

Cluster 0: Sports
   Category                                            Content  \
42   Sports  Donovan Mitchell’s 39 points have led hosts Cl...   
43   Sports  Anthony Edwards scored 43 points, Karl-Anthony...   
44   Sports  The top-seeded Boston Celtics and Oklahoma Cit...   
45   Sports  Malaysia’s Football Association has recommende...   
46   Sports  Real Madrid earned what coach Carlo Ancelotti ...   

                                                 Link      Source  \
42  https://www.aljazeera.com/sports/2024/5/6/nba-...  Al Jazeera   
43  https://www.aljazeera.com/sports/2024/5/5/nba-...  Al Jazeera   
44  https://www.aljazeera.com/sports/2024/5/8/nba-...  Al Jazeera   
45  https://www.aljazeera.com/sports/2024/5/8/mala...  Al Jazeera   
46  https://www.aljazeera.com/sports/2024/5/5/la-l...  Al Jazeera   

                                                Title  Unnamed: 5  Cluster  
42  Cleveland Cavaliers in Eastern Conference semi...         NaN        0  
43  Minnesota st